### DM-04 Assignment 1  

#### Import libraries  

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss

#### Parameters  

In [2]:
csv_in = 'shop_sales.csv'

#### Read CSV file  

In [3]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

(200, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ShopID  200 non-null    int64 
 1   Pref    200 non-null    object
 2   Type    200 non-null    object
 3   Sales1  200 non-null    int64 
 4   Sales2  200 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.9+ KB
None


,ShopID,Pref,Type,Sales1,Sales2
0,101,Saitama,franchise,108,110
1,102,Saitama,direct,94,108
2,103,Tokyo,direct,103,89
3,104,Saitama,franchise,99,122
4,105,Kanagawa,franchise,113,109


#### 1. Is Sales2 larger than Sales1?  
Welch (t) test for difference of poppulation means (paired t-test, one-sided)  
母平均の差の検定(対応のあるWelch(t)検定)  

In [4]:
t, p = ss.ttest_1samp(df['Sales2']-df['Sales1'], 0.0)
print(t, p/2)

9.227723073665455 2.0675782068478105e-17


**Answer 1:  
H0: Sales2の母平均とSales1の母平均に差はない  
H1: Sales2の母平均はSales1の母平均より大  
結論: Sales2の母平均はSales1の母平均より有意に大  
根拠: t > 0 and p/2 = 2.068e-17 < 0.05**

**H0: There is no difference between the population mean of Sales2 and that of Sales1  
H1: The population mean of Sales2 is larger than that of Sales1  
Conclusion: The population mean of Sales2 is significantly larger than that of Sales1  
Basis: t > 0 and p/2 = 2.068e-17 < 0.05**

#### 2. Are Sales2 of Tokyo and Saitama different?  
Welch (t) test for difference of poppulation means (unpaired t-test, two-sided)  
TokyoとSaitamaのSales2の販売数は、有意に異なるといえるか。  

In [5]:
df_tky = df[ df['Pref'] == 'Tokyo' ]
df_stm = df[ df['Pref'] == 'Chi' ]
t, p = ss.ttest_ind(df_tky['Sales2'], df_stm['Sales2'], equal_var=False)
print(t, p)

0.11753155786638277 0.9066597833906881


**Answer 2:  
H0: Tokyoの母平均とSaitamaの母平均に差はない  
H1: Tokyoの母平均とSaitamaの母平均は異なる  
結論: Tokyoの母平均とSaitamaの母平均は有意に異なるとはいえない  
根拠: p = 0.907 > 0.05**

**H0: There is no difference between the population mean of Tokyo and that of Saitama  
H1: The population mean of Tokyo is different from that of Saitama  
Conclusion: We can not say that there is significant difference between the population mean of Tokyo and that of Saitama.  
Basis: p = 0.907 > 0.05**

#### 3. Is there any relationship between Pref and Shop types?  
(test of independence by chi-squared test)   
都道府県と店舗種類のクロス集計表を作成し、都道府県と店舗種類の間に関連があるかどうかを判断  
(独立性のカイ2乗検定)  

In [6]:
ct = pd.crosstab(df['Pref'], df['Type'])
display(ct)

Type,direct,franchise
Pref,,
Chiba,13,15
Kanagawa,28,30
Saitama,24,25
Tokyo,34,31


In [7]:
chi2, p, dof, expected = ss.chi2_contingency(ct)
print(chi2, p, dof, expected)

0.35072743456882416 0.9502219446727902 3 [[13.86  14.14 ]
 [28.71  29.29 ]
 [24.255 24.745]
 [32.175 32.825]]


**Answer 3:  
H0: 都道府県と店舗種類の間には関連がない（両者は独立である）  
H1: 都道府県と店舗種類の間には関連がある（両者は独立でない）  
結論: 都道府県と店舗種類の間には有意な関連があるとはいえない（この都道府県はこの種類が多い、などの傾向はない）  
根拠: p = 0.950 > 0.05**

**H0: There is no relationship between Prefs and Store types (independend).  
H1: There is a relationship between Prefs and Store types (dependent).  
Conclusion: We can not say that there is a significant relationship between Prefs and Store types.  
Basis: p = 0.950 > 0.05**

#### (Adv) 4. Is the ratio of #shops in each pref consistent to the ratio of population of each pref?  
(M: 1 milion; Tokyo:14.0M, Saitama:7.3M, Chiba:6.3M, Kanagawa:9.2M)  
(発展) 各都道府県の店舗数の比率は、都道府県の人口(Mを100万人として、  
Tokyo:14.0M, Saitama:7.3M, Chiba:6.3M, Kanagawa:9.2M)  の比率  
と一致しているとみなしてよいか。  

chi-square goodness-of-fit test  
適合度のカイ2乗検定   

In [8]:
shop_freq = df['Pref'].value_counts()
n_shops = df['Pref'].value_counts().sum()
print(n_shops)
print(shop_freq)

200
Tokyo       65
Kanagawa    58
Saitama     49
Chiba       28
Name: Pref, dtype: int64


In [9]:
pop = pd.Series({'Tokyo':14.0, 'Saitama':7.3, 'Chiba':6.3, 'Kanagawa':9.2}, name='pop')
print(pop)

Tokyo       14.0
Saitama      7.3
Chiba        6.3
Kanagawa     9.2
Name: pop, dtype: float64


In [10]:
pop_ratio = pop / pop.sum()
print(pop_ratio)

Tokyo       0.380435
Saitama     0.198370
Chiba       0.171196
Kanagawa    0.250000
Name: pop, dtype: float64


#### Make expected #shops according to pop_ratio  

In [11]:
shop_exp = pop_ratio * n_shops
shop_exp = shop_exp[ shop_freq.index ]
print(shop_exp)

Tokyo       76.086957
Kanagawa    50.000000
Saitama     39.673913
Chiba       34.239130
Name: pop, dtype: float64


In [12]:
chi2, p = ss.chisquare(shop_freq, f_exp=shop_exp)
print(chi2, p)

6.224705370732769 0.10117516999032702


**Answer 4:  
H0: 比率は一致している  
H1: 比率は異なっている  
結論: 比率が有意に異なっているとはいえない    
根拠: p = 0.101 > 0.05**  

**H0: Ratios are the same.  
H1: Ratios are different from each other.  
Conclusion: We can not say that ratios are significantly different from each other.  
Basis: p = 0.101 > 0.05**